In [1]:
%matplotlib inline

import stumpy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt
import dtale

import wrangle as wg
import scipy.stats as stats

plt.rcParams["figure.figsize"] = [20,6]  # width, height
plt.rcParams['xtick.direction'] = 'out'

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = wg.get_dow()
df.head()

,quarter,stock,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,AA,1/7/2011,$15.82,$16.72,$15.78,$16.42,239655616,3.79267,NaN,NaN,$16.71,$15.97,-4.428490,26,0.182704
1,1,AA,1/14/2011,$16.71,$16.71,$15.64,$15.97,242963398,-4.42849,1.380223,239655616.0,$16.19,$15.79,-2.470660,19,0.187852
2,1,AA,1/21/2011,$16.19,$16.38,$15.60,$15.79,138428495,-2.47066,-43.024959,242963398.0,$15.87,$16.13,1.638310,12,0.189994
3,1,AA,1/28/2011,$15.87,$16.63,$15.82,$16.13,151379173,1.63831,9.355500,138428495.0,$16.18,$17.14,5.933250,5,0.185989
4,1,AA,2/4/2011,$16.18,$17.39,$16.18,$17.14,154387761,5.93325,1.987452,151379173.0,$17.33,$17.37,0.230814,97,0.175029


In [3]:
df = wg.prep_DOW(df)
df.head()

,quarter,stock,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
date,,,,,,,,,,,,,,,
2011-01-04,2,IBM,161.54,164.42,161.12,164.27,19081936,1.68998,-24.820375,25381792.0,164.42,164.05,-0.225033,35,0.456565
2011-01-04,2,TRV,59.09,60.01,58.37,59.71,16608737,1.04925,17.569654,14126721.0,59.83,59.58,-0.417851,68,0.686652
2011-01-04,2,KO,65.20,67.48,65.03,67.22,39265955,3.09816,31.529107,29853434.0,67.22,67.27,0.074383,73,0.699197
2011-01-04,2,XOM,83.86,84.94,82.61,84.68,77569311,0.97782,-6.951645,83364516.0,84.26,85.95,2.005700,40,0.555031
2011-01-04,2,PG,61.08,62.30,60.61,62.08,43453701,1.63720,5.084613,41351155.0,62.17,61.90,-0.434293,26,0.853737


In [4]:
# top 1 for price of stock / # 2 for percent change mean
df_ibm = df.where(df['stock'] == 'IBM')
df_ibm = df_ibm.dropna()

# top 1 for % change mean
df_pfe = df.where(df['stock'] == 'PFE')
df_pfe = df_pfe.dropna()

# top 1 for volume
df_bac = df.where(df['stock'] == 'BAC')
df_bac = df_bac.dropna()


# frankly, not enough data was in this DF but alas, we'll see how it goes
df_ibm.shape, df_pfe.shape, df_bac.shape

((25, 15), (25, 15), (25, 15))

In [5]:
# set train size to be 80% of total 
train_size = int(round(df_ibm.shape[0] * 0.8))
train_size

20

In [6]:
# set test size to be number of rows remaining. 
test_size = int(round(df_ibm.shape[0] * 0.2))
test_size

5